# 2023 해류, 해안쓰레기, 선박밀집도

In [2]:

import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# 파일 경로 설정
tidal_current_path = "tidal_current_2023.csv"
marideb_timestep_path = "marideb_timestep.csv"
grid_path = "grid_latlon.csv"
debris_path = "marideb_location.csv"

# 데이터 로드
tidal_current_df = pd.read_csv(tidal_current_path)   # 해류 데이터
marideb_timestep_df = pd.read_csv(marideb_timestep_path)  # 선박 밀집도 데이터
deb_df = pd.read_csv(debris_path) #해안쓰레기 데이터
grid_df = pd.read_csv(grid_path)   # 기존 격자 데이터


In [3]:
print(grid_df.head())  # Grid_ID, Latitude, Longitude 존재 확인


       Grid_ID  Latitude  Longitude
0  GR3_G3B32_D    35.475    129.425
1  GR3_G3B33_P    35.075    129.025
2  GR3_G3B33_L    35.125    129.075
3  GR3_F4H23_E    34.725    127.725
4  GR3_G3E12_S    34.825    128.425


In [4]:
# 기존 격자의 위경도 좌표 추출
grid_points = grid_df[["Latitude", "Longitude"]].values

# 쓰레기 데이터의 위경도 좌표
deb_points = deb_df[["위도", "경도"]].values

tree = cKDTree(grid_points)
_, indices = tree.query(deb_points)


deb_df["Grid_ID"] = grid_df.iloc[indices]["Grid_ID"].values


In [5]:
# 기존 격자의 위경도 좌표 추출
grid_points = grid_df[["Latitude", "Longitude"]].values

# 해류 데이터의 위경도 좌표
tidal_points = tidal_current_df[["pre_lat", "pre_lon"]].values


# 최근접 격자 찾기 (cKDTree 사용)
tree = cKDTree(grid_points)
_, indices = tree.query(tidal_points)


# 해류 데이터에 매핑된 격자 정보 추가
tidal_current_df["Grid_ID"] = grid_df.iloc[indices]["Grid_ID"].values

In [6]:
# 격자별 평균 해류 속도 및 방향 계산
tidal_grid = tidal_current_df.groupby("Grid_ID").agg(
    avg_speed=("current_speed", "mean"),
    avg_dir=("current_dir", "mean")
).reset_index()

# 기존 격자와 병합하여 저장
grid_with_tidal = grid_df.merge(tidal_grid, on="Grid_ID", how="left")

# 결측값 처리 (보간 또는 평균 대체)
grid_with_tidal["avg_speed"] = grid_with_tidal["avg_speed"].fillna(0)  # 속도가 없는 곳은 0
grid_with_tidal["avg_dir"] = grid_with_tidal["avg_dir"].fillna(0)  # 방향 없는 곳도 0


In [7]:
# 결과 데이터 저장
output_path = "grid_with_avg_tidal_2023.csv"
grid_with_tidal.to_csv(output_path, index=False)

print(f"격자별 평균 해류 데이터가 저장되었습니다: {output_path}")


격자별 평균 해류 데이터가 저장되었습니다: grid_with_avg_tidal_2023.csv


In [8]:
# 필요한 컬럼만 선택
grid_with_tidal_filtered = grid_with_tidal[["Grid_ID", "Latitude", "Longitude", "avg_speed", "avg_dir"]]

# 결과 저장
output_filtered_path = "grid_with_avg_tidal_filtered_2023.csv"
grid_with_tidal_filtered.to_csv(output_filtered_path, index=False)

print(f"필요한 컬럼만 저장된 파일: {output_filtered_path}")


필요한 컬럼만 저장된 파일: grid_with_avg_tidal_filtered_2023.csv


In [9]:
# deb_df 데이터프레임을 CSV 파일로 저장
deb_df.to_csv("deb_df_output_2023.csv", index=False)

print("deb_df 데이터가 'deb_df_output.csv'로 저장되었습니다.")


deb_df 데이터가 'deb_df_output.csv'로 저장되었습니다.


In [10]:
import pandas as pd
from scipy.spatial import cKDTree

# 파일 경로 설정
tidal_path = "tidal_current_2023.csv"  # 해류 데이터 (위경도, 속도, 방향, 시간 포함)
grid_path = "grid_latlon.csv"  # 기존 그리드 데이터 (Grid_ID, 위경도)

# 데이터 로드
tidal_df = pd.read_csv(tidal_path)  # 해류 데이터
grid_df = pd.read_csv(grid_path)  # 격자 데이터

# 데이터 확인
print(tidal_df.head())
print(grid_df.head())


   current_dir    pre_lon  current_speed   pre_lat             datetime
0          147  125.03305           43.0  33.04701  2023-01-01 00:00:00
1          146  125.02657           44.0  33.19098  2023-01-01 00:00:00
2          147  125.02003           45.0  33.33496  2023-01-01 00:00:00
3          147  125.01345           43.0  33.47892  2023-01-01 00:00:00
4          147  125.00682           42.0  33.62288  2023-01-01 00:00:00
       Grid_ID  Latitude  Longitude
0  GR3_G3B32_D    35.475    129.425
1  GR3_G3B33_P    35.075    129.025
2  GR3_G3B33_L    35.125    129.075
3  GR3_F4H23_E    34.725    127.725
4  GR3_G3E12_S    34.825    128.425


In [11]:
# 기존 격자의 위경도 좌표 추출
grid_points = grid_df[["Latitude", "Longitude"]].values

# 해류 데이터의 위경도 좌표
tidal_points = tidal_df[["pre_lat", "pre_lon"]].values

# 최근접 격자 찾기
tree = cKDTree(grid_points)
_, indices = tree.query(tidal_points)

# 해류 데이터에 매핑된 Grid_ID 추가
tidal_df["Grid_ID"] = grid_df.iloc[indices]["Grid_ID"].values


In [12]:
# 시간 컬럼을 datetime 타입으로 변환
tidal_df["datetime"] = pd.to_datetime(tidal_df["datetime"])

# Grid_ID 및 시간별 평균 해류 속도 및 방향 계산
grid_tidal_hourly = tidal_df.groupby(["Grid_ID", "datetime"]).agg(
    avg_speed=("current_speed", "mean"),
    avg_dir=("current_dir", "mean")
).reset_index()

# 결과 확인
print(grid_tidal_hourly.head())


       Grid_ID            datetime  avg_speed  avg_dir
0  GR3_F2J31_N 2023-01-01 00:00:00       63.0    307.0
1  GR3_F2J31_N 2023-01-01 01:00:00       82.0    294.0
2  GR3_F2J31_N 2023-01-01 02:00:00       84.0    284.0
3  GR3_F2J31_N 2023-01-01 03:00:00       67.0    271.0
4  GR3_F2J31_N 2023-01-01 04:00:00       41.0    243.0


In [13]:
# CSV 파일로 저장
output_path = "grid_tidal_hourly_2023.csv"
grid_tidal_hourly.to_csv(output_path, index=False)

print(f"그리드별 시간당 해류 데이터가 저장되었습니다: {output_path}")


그리드별 시간당 해류 데이터가 저장되었습니다: grid_tidal_hourly_2023.csv
